<a href="https://colab.research.google.com/github/inderpreetsingh01/PyMath/blob/main/Pandas_SQL_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [43]:
# q1. Find all numbers that appear at least three times consecutively.
def consecutive_numbers1(logs: pd.DataFrame, min_consecutive) -> pd.DataFrame:
    rolling_windows = logs['num'].rolling(min_consecutive)
    mask = rolling_windows.var() == 0
    unique_nums = logs[mask]['num'].unique()
    return pd.DataFrame({"ConsecutiveNums":[] if not unique_nums else unique_nums})

def consecutive_numbers2(logs: pd.DataFrame, min_consecutive) -> pd.DataFrame:
    rolling_windows = logs['num'].rolling(min_consecutive)
    mask = rolling_windows.apply(lambda x:x.nunique()==1)==1
    unique_nums = logs[mask]['num'].unique()
    return pd.DataFrame({"ConsecutiveNums":[] if not unique_nums else unique_nums})


# SQL Solution
# with cte as (
#     select num,
#     lead(num,1) over() num1,
#     lead(num,2) over() num2
#     from logs
# )
# select distinct num as ConsecutiveNums
# from cte
# where num1=num2 and num=num1

# SELECT DISTINCT a.num
# FROM numbers_table a
# JOIN numbers_table b ON a.num = b.num AND b.id = a.id + 1
# JOIN numbers_table c ON a.num = c.num AND c.id = a.id + 2;

In [42]:
logs = pd.DataFrame({"id":np.arange(1, 8), "num":[1,1,1,2,1,2,2]})
min_consecutive = 3
consecutive_numbers2(logs, min_consecutive)

,ConsecutiveNums
0,1


In [ ]:
# q2 Write a solution to find the dense rank of the scores.
def order_scores(scores: pd.DataFrame) -> pd.DataFrame:
    # Solution 1
    if not len(scores):
        return pd.DataFrame({"score":[], "rank":[]})
    sorted_scores = scores['score'].sort_values(ascending=False)
    rank = 1
    ranks = [1]
    for i in range(1, len(sorted_scores)):
        if sorted_scores.iloc[i]!=sorted_scores.iloc[i-1]:
            rank+=1
        ranks.append(rank)
    return pd.DataFrame({"score":sorted_scores.values, "rank":ranks})

    # Solution 2
    # scores['rank'] = scores['score'].rank(method='dense', ascending=False)
    # result_df = scores.drop('id', axis=1).sort_values(by='score', ascending=False)
    # return result_df

    # Solution 3
    # scores = scores.sort_values(by='score', ascending=False)
    # scores['rank'] = (~(scores['score'].duplicated())).cumsum()
    # scores = scores.drop('id', axis=1)
    # return scores

    # Solution 4
    # scores_unique = np.sort(scores.score.unique())[::-1]
    # dic = {score:i+1 for i, score in enumerate(scores_unique)}
    # scores['rank'] = scores['score'].map(dic)
    # scores = scores.drop('id', axis=1)
    # scores = scores.sort_values(by='score', ascending=False)
    # return scores


# sql solution
# with cte as(
#     SELECT
#         e1.score,
#         COUNT(DISTINCT e2.score) AS dense_rank_
#     FROM
#         scores e1
#     JOIN
#         scores e2 ON e2.score >= e1.score
#     GROUP BY
#         e1.score
#     ORDER BY
#         dense_rank_
# )
# select s.score, cte.dense_rank_ as `rank`
# from scores s
# join cte on cte.score=s.score
# order by s.score desc

In [ ]:
# q3 Write a solution to find the nth highest distinct salary from the Employee table. If there are less than n distinct salaries, return null.
def nth_highest_salary(employee: pd.DataFrame, N: int) -> pd.DataFrame:
    sorted_salaries = employee['salary'].sort_values(ascending=False).drop_duplicates()
    return pd.DataFrame({f"getNthHighestSalary({N})":[None if len(sorted_salaries)<N or N<=0 else sorted_salaries.iloc[N-1]]})

# sql solution
# CREATE FUNCTION getNthHighestSalary(N INT) RETURNS INT
# BEGIN
#   RETURN (
#     with cte as (
#         select salary, dense_rank() over(order by salary desc) as rnk
#         from Employee
#     )
#     select
#         case when (max(rnk)>=N) then (select min(salary) from cte where rnk=N) end as getNthHighestSalary
#         from cte
#   );
# END

In [ ]:
# q5 Department Highest Salary
def department_highest_salary(employee: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:

    # Common
    employee = employee.rename(columns={'name':'Employee', 'salary':'Salary'})
    department = department.rename(columns={'name':'Department'})
    df = pd.merge(employee, department, left_on='departmentId', right_on='id')
    df = df[['Department', 'Employee', 'Salary']]

    # Solution 1 (apply)
    def func(df):
        mask = df.Salary==max(df.Salary)
        return df.loc[mask]

    return df.groupby('Department').apply(func)

    # Solution 2 (transform)
    max_per_dept = df.groupby('Department')['Salary'].transform('max')
    mask = df['Salary']==max_per_dept
    return df[mask]


# sql solution
# with cte as(
#     select departmentId, max(salary) max_salary
#     from employee
#     group by departmentId
# )
# select d.name as Department, e.name as Employee, e.salary Salary
# from employee e
# join department d on d.id=e.departmentId
# join cte on cte.departmentId=d.id
# where e.salary=cte.max_salary

In [ ]:
# q6 Game Play Analysis IV
# Write a solution to report the fraction of players that logged in again on the day after the day they first logged in, rounded to 2 decimal places.
# In other words, you need to determine the number of players who logged in on the day immediately following their initial login, and divide it by the number of total players.
def gameplay_analysis(activity: pd.DataFrame) -> pd.DataFrame:
    def func(df):
        s = (df.event_date - df.event_date.shift(1)).dt.days.unique()
        return 1 in s

    activity['start_date'] = activity.groupby('player_id').event_date.transform(min)
    activity_2nd_day = activity.loc[activity['start_date'] + pd.DateOffset(1) == activity["event_date"]]
    total_players = activity.player_id.nunique()
    return pd.DataFrame({'fraction':[round(len(activity_2nd_day)/total_players, 2)]})


# sql solution
# with cte as(
#     select player_id, event_date,
#     min(event_date) over(partition by player_id) as min_date
#     from activity
# ),
# cte1 as (
#     select player_id, datediff(event_date, min_date) as date_diff
#     from cte
# )
# select round((select count(distinct player_id)
# from cte1
# where date_diff=1)/count(distinct player_id), 2) as fraction
# from activity

# solution 2
# select
#   round(
#       count(distinct case when datediff(event_date,min_date)=1 then player_id end)/
#       count(distinct player_id), 2) as fraction
#   from (
#       select event_date, player_id,
#       min(event_date) over(partition by player_id) as min_date
#       from activity
#   ) as t

In [ ]:
# q7 Write a solution to find managers with at least five direct reports.

def find_managers(employee: pd.DataFrame) -> pd.DataFrame:
    df = employee['managerId'].value_counts()
    manager_ids = df[df>=5].index.values
    employee_names = employee.loc[employee['id'].isin(manager_ids)][['name']]
    return employee_names

# sql solution
# select name from employee
# where id in
#     (   select managerId
#         from employee
#         group by managerId
#         having count(*)>=5
#     )

In [ ]:
# q8 Investments in 2016
# Write a solution to report the sum of all total investment values in 2016 tiv_2016, for all policyholders who:
# have the same tiv_2015 value as one or more other policyholders, and
# are not located in the same city as any other policyholder (i.e., the (lat, lon) attribute pairs must be unique).
# Round tiv_2016 to two decimal places.

def find_investments(insurance: pd.DataFrame) -> pd.DataFrame:
    # Solution 1
    # tiv_count = insurance['tiv_2015'].value_counts().to_dict()
    # lat_lon_count = insurance[['lat', 'lon']].value_counts().to_dict()
    # insurance['tiv_count'] = insurance['tiv_2015'].map(tiv_count)
    # insurance['lat_lon_count'] = insurance.apply(lambda row:lat_lon_count[(row['lat'], row['lon'])], axis=1)
    # final_ans = insurance[(insurance['tiv_count']>1) & (insurance['lat_lon_count']==1)][['tiv_2016']].sum()
    # return pd.DataFrame({"tiv_2016":round(final_ans, 2)})

    # Solution 2
    dup = insurance[insurance.duplicated(subset = 'tiv_2015', keep = False)].pid
    pos = insurance[~insurance.duplicated(subset = ['lat', 'lon'], keep=False)].pid
    return pd.DataFrame({'tiv_2016':[round(insurance[insurance.pid.isin(pos) & insurance.pid.isin(dup)].tiv_2016.sum(), 2)]})


# sql solution
# with cte as (
# select tiv_2016,
#     count(*) over(partition by tiv_2015) as count_tiv_2015,
#     count(*) over(partition by lat, lon) as lat_lon_count
#     from insurance
# )
# select round(sum(tiv_2016), 2) tiv_2016
# from cte
# where count_tiv_2015>1 and lat_lon_count=1

In [ ]:
# q9 Friend Requests II: Who Has the Most Friends LC602
def most_friends(request_accepted: pd.DataFrame) -> pd.DataFrame:
    # Solution 1
    series = pd.concat([request_accepted['requester_id'], request_accepted['accepter_id']]).value_counts()
    return series[series==max(series)].reset_index().rename(columns = {"index":
    "id", "count":"num"})

    # Solution 2
    res = pd.concat([request_accepted["requester_id"], request_accepted["accepter_id"]]).tolist()
    r = mode(res)
    return pd.DataFrame({"id" : [r], "num" : [res.count(r)]})


# sql solution

# solution 1 simple
# select id, count(*) num
# from (
#     select requester_id as id from RequestAccepted
#     union all
#     select accepter_id as id from RequestAccepted
# ) f
# group by id
# order by num desc
# limit 1

# solution 2
# WITH requester_counts AS (
#     SELECT requester_id AS id, COUNT(*) AS cnt
#     FROM RequestAccepted
#     GROUP BY requester_id
# ),
# accepter_counts AS (
#     SELECT accepter_id AS id, COUNT(*) AS cnt
#     FROM RequestAccepted
#     GROUP BY accepter_id
# ),
# combined_counts AS (
#     SELECT
#         COALESCE(r.id, a.id) AS id,                      # this is how to select between null and actual number, coalesce returns first non null values,
#         COALESCE(r.cnt, 0) + COALESCE(a.cnt, 0) AS num.  # you were using case to select a.id if r.id is null case can't be applied directly as such
#     FROM requester_counts r
#     OUTER JOIN accepter_counts a ON r.id = a.id
# )
# SELECT id, num
# FROM combined_counts
# WHERE num = (SELECT MAX(num) FROM combined_counts);      # this is how where is applied on aggregates

In [ ]:
# q10 Tree Node

# solution 1
def tree_node(tree: pd.DataFrame) -> pd.DataFrame:
    tree.loc[tree['p_id'].isna(), 'type'] = 'Root'
    tree.loc[(tree['id'].isin(tree['p_id'].unique())) & (tree['type'].isna()), 'type'] = 'Inner'
    tree.loc[tree['type'].isna(), 'type'] = 'Leaf'
    return tree[['id', 'type']]

# solution 2
# cleaner solution
def tree_node(tree: pd.DataFrame) -> pd.DataFrame:
    # Create a copy to avoid modifying input
    result = tree.copy()

    # Get all unique parent ids (excluding null)
    parent_ids = tree['p_id'].dropna().unique()

    # Initialize type column
    result['type'] = 'Leaf'  # Default case

    # Set Root nodes (where p_id is null)
    result.loc[tree['p_id'].isna(), 'type'] = 'Root'

    # Set Inner nodes (nodes that are parents of others)
    result.loc[tree['id'].isin(parent_ids), 'type'] = 'Inner'

    return result[['id', 'type']]


# sql solution
# select id,
#     case
#         when p_id is null then 'Root'
#         when id in (select distinct p_id from tree where p_id is not null) then 'Inner'
#         else 'Leaf'
#         end
#     as type
# from tree

In [ ]:
# q11